###### -----------------START--------------------------------------------

In [1]:
import json

In [2]:
import os

In [3]:
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt

In [5]:
train_file_path = '/home/aritra/cric/train_questions.json'
val_file_path = '/home/aritra/cric/val_questions.json'
test_file_path = '/home/aritra/cric/test_v1_questions.json'

In [6]:
# Training Set

with open(train_file_path, "r") as file:
     train_json = json.load(file)

In [7]:
# Validation Set

with open(val_file_path, "r") as file:
     val_json = json.load(file)

In [8]:
# Test Set

with open(test_file_path, "r") as file:
     test_json = json.load(file)

In [9]:
len(train_json)

365235

In [10]:
len(val_json)

43112

In [11]:
len(test_json)

86003

In [12]:
train_json[1099]['question']

'which brown animal walking in the field could be used for transporting people'

In [13]:
val_json[1099]['question']

'is there an object that is a type of public transports'

In [14]:
test_json[1099]['question']

'can the ceramic bird spread wings'

### ------------------------------Extracting Data of Training Set-------------------------------------------------------------------------------



In [15]:
questionList = []
answerList = []
imgList = []

In [16]:
train_json[2]['image_id']

1005

#### iter 1: from 0 , 149000 -> error1.txt -> 159
#### iter 2: from 150000 , 240000 -> error2.txt -> 34
#### iter 3: from 240000 , 365235 ->error3.txt -> 121

In [17]:
# verifying
indexToExclude = []

with open('error1.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExclude.append(number)
        
with open('error2.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExclude.append(number)
        
with open('error3.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExclude.append(number)

In [18]:
len(indexToExclude)

314

In [19]:
for i in tqdm(range(len(train_json))):
    
    if i in indexToExclude:
        continue
        
    pointer = train_json[i]
    
    questionList.append(pointer['question'])
    answerList.append(pointer['answer'])
    imgList.append(pointer['image_id'])

100%|███████████████████████████████████| 365235/365235 [00:01<00:00, 319729.85it/s]


In [20]:
len(questionList), len(answerList), len(imgList)

(364921, 364921, 364921)

In [21]:
len(list(set(answerList)))

1442

### ---------------------------------------Map Creation--------------------------------------------------------

In [22]:
def findUnique(targetList):
    
    uniqueList = []
    
    for word in targetList:
        if word not in uniqueList:
            uniqueList.append(word)
    
    return uniqueList

In [23]:
len(findUnique(answerList))

1442

In [24]:
# creating word to number mapping

mapping = {}
counter = 0

uniqueAnsList = findUnique(answerList)

for word in uniqueAnsList:
    
    if word not in mapping:
        
        mapping[word] = counter
        counter += 1

In [25]:
uniqueAnsList[0:5]

['no', 'small', 'picture', 'table', 'bookshelf']

In [26]:
numOfClasses = max(mapping.values())
numOfClasses

1441

In [27]:
len(mapping)

1442

In [28]:
# creating number to word mapping

reverse_mapping = dict([(value, key) for key, value in mapping.items()])

### --------------------------------------Processing of Training Set--------------------------------------------------------------------

In [29]:
labels = []

for i in range(len(answerList)):
    labels.append( mapping[ answerList[i] ] )

In [30]:
len(labels)

364921

In [31]:
scores = []

for i in tqdm(range(len(answerList))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList[i]] ] = 1
    
    scores.append(s)

100%|████████████████████████████████████| 364921/364921 [00:03<00:00, 91805.66it/s]


In [32]:
len(scores)

364921

In [33]:
imgPathList = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList))):
    
    imgName = str(imgList[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList.append(concatedPath)

100%|███████████████████████████████████| 364921/364921 [00:00<00:00, 767628.01it/s]


In [34]:
from datasets import load_dataset
from datasets import Dataset
import datasets
from PIL import Image
import torch

In [35]:
imgPathList[0:5]

['/home/aritra/cric/images/img/1000.jpg',
 '/home/aritra/cric/images/img/1005.jpg',
 '/home/aritra/cric/images/img/1005.jpg',
 '/home/aritra/cric/images/img/1005.jpg',
 '/home/aritra/cric/images/img/1008.jpg']

In [36]:
len(imgPathList)

364921

In [37]:
listToDictionary = {'questions':questionList, 'labels': labels, 'scores': scores, 'images':imgPathList}
modified_train_set = Dataset.from_dict(listToDictionary)

In [38]:
# mapping each filepath to images in the directory

modified_train_set = modified_train_set.cast_column("images", datasets.Image())

In [39]:
modified_train_set

Dataset({
    features: ['questions', 'labels', 'scores', 'images'],
    num_rows: 364921
})

## -----------------------------------Extracting Validation Set---------------------------------------------

In [40]:
questionList_val = []
answerList_val = []
imgList_val = []

In [41]:
# collecting the index containing errorneous images

indexToExcludeVal = []
with open('error_validation.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExcludeVal.append(number)

with open('error_validation2.txt', 'r') as file:
    for line in file:
        number = int(line.strip())  # Convert the read line to an integer
        indexToExcludeVal.append(number)


In [42]:
# excluding the index containing errorneous images

for i in tqdm(range(len(val_json))):
    
    if (i in indexToExcludeVal):
        continue
        
    pointer = val_json[i]
    
    questionList_val.append(pointer['question'])
    answerList_val.append(pointer['answer'])
    imgList_val.append(pointer['image_id'])

100%|██████████████████████████████████████| 43112/43112 [00:02<00:00, 16205.99it/s]


43112 -> 43068 -> 33175

In [43]:
len(questionList_val), len(answerList_val), len(imgList_val)

(33175, 33175, 33175)

In [44]:
uniqueAnswerListVal = list(set(answerList_val))
len(uniqueAnswerListVal)

266

In [45]:
# check if all the uniques answers are present in the mapping

y,n = 0,0
store = []
for i in range(len(answerList_val)):
    
    word = answerList_val[i]
    
    if word in mapping:
        y += 1
    else:
        n+=1
        store.append(i)

In [46]:
y

33175

### --------------------------------------------------------Processing Validation Set-------------------------------------------------------

In [47]:
labels_val = []

for i in range(len(answerList_val)):
    labels_val.append( mapping[ answerList_val[i] ] )

In [48]:
len(labels_val)

33175

In [49]:
scores_val = []

for i in tqdm(range(len(answerList_val))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList_val[i]] ] = 1
    
    scores_val.append(s)

100%|██████████████████████████████████████| 33175/33175 [00:00<00:00, 92510.18it/s]


In [50]:
len(scores_val)

33175

In [51]:
imgPathList_val = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList_val))):
    
    imgName = str(imgList_val[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList_val.append(concatedPath)

100%|█████████████████████████████████████| 33175/33175 [00:00<00:00, 762001.44it/s]


In [52]:
imgPathList_val[0:5]

['/home/aritra/cric/images/img/1003.jpg',
 '/home/aritra/cric/images/img/1003.jpg',
 '/home/aritra/cric/images/img/1018.jpg',
 '/home/aritra/cric/images/img/1018.jpg',
 '/home/aritra/cric/images/img/1027.jpg']

In [53]:
# creating HF dataset to map images fast of Val_set

listToDictionary = {'questions':questionList_val, 'labels':labels_val, 'scores':scores_val, 'images':imgPathList_val}
modified_val_set = Dataset.from_dict(listToDictionary)

In [54]:
# mapping each filepath of Val Set to images in the directory

modified_val_set = modified_val_set.cast_column("images", datasets.Image())

### -------------------------------------------Extracting Color Questions Set-------------------------------------------------


In [55]:
# color questions of the train set is stored in this file.
# objective is to train a fresh model on only color based questions

indices = []
with open('./text_files/color_questions_indices_train_set.txt', 'r') as file:
    for number in file:
        number = int(number.strip())
        indices.append(number)


In [56]:
indices[0:5]

[0, 1, 16, 29, 32]

In [57]:
questionList_color = []
answerList_color = []
imgList_color = []

In [58]:
# from the predefined list of color indices of the train set here the questions,answer,img is being copied 

for i in indices:
                
    questionList_color.append( questionList[i] )
    answerList_color.append( answerList[i] )
    imgList_color.append( imgList[i] )

In [59]:
questionList_color[10:15]

['which green thing near the green house could be opened or closed',
 'what type of object is on back of the tan object that I can use for sitting on',
 'can the black electronic device that is wearing the hand control small electrical appliance',
 'which color is the object that is on back of the seat and is a type of electronic device',
 'is there a yellow vehicle that can travel on road']

In [60]:
len(questionList_color), len(answerList_color), len(imgList_color)

(149163, 149163, 149163)

In [61]:
uniqueAnswerListColor = list(set(answerList_color))
len(uniqueAnswerListColor)

1065

In [62]:
# check if all the uniques answers are present in the mapping

y,n = 0,0
store = []
for i in range(len(answerList_color)):
    
    word = answerList_color[i]
    
    if word in mapping:
        y += 1
    else:
        n+=1
        store.append(i)

In [63]:
y

149163

In [64]:
# Processing of color set

labels_color = []

for i in range(len(answerList_color)):
    labels_color.append( mapping[ answerList_color[i] ] )

In [65]:
len(labels_color)

149163

In [66]:
scores_color = []

for i in tqdm(range(len(answerList_color))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList_color[i]] ] = 1
    
    scores_color.append(s)

100%|████████████████████████████████████| 149163/149163 [00:01<00:00, 90223.22it/s]


In [67]:
len(scores_color)

149163

In [68]:
imgPathList_color = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList_color))):
    
    imgName = str(imgList_color[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList_color.append(concatedPath)

100%|███████████████████████████████████| 149163/149163 [00:00<00:00, 769226.47it/s]


In [69]:
len(imgPathList_color)

149163

In [70]:
imgPathList_val[0:5]

['/home/aritra/cric/images/img/1003.jpg',
 '/home/aritra/cric/images/img/1003.jpg',
 '/home/aritra/cric/images/img/1018.jpg',
 '/home/aritra/cric/images/img/1018.jpg',
 '/home/aritra/cric/images/img/1027.jpg']

In [71]:
# creating HF dataset to map images fast of Val_set

listToDictionary = {'questions':questionList_color, 'labels':labels_color, 'scores':scores_color, 'images':imgPathList_color}
train_color_set = Dataset.from_dict(listToDictionary)

In [72]:
# mapping each filepath of Val Set to images in the directory

train_color_set = train_color_set.cast_column("images", datasets.Image())

### -------------------------------------------Extracting Test Set-------------------------------------------------


In [73]:
questionList_test = []
answerList_test = []
imgList_test = []

In [74]:
indexToExcludeTest = []

with open('error_testSet1.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExcludeTest.append(number)
        
with open('errorTestSet2.txt', 'r') as file:
    for line in file:
        number = int(line.strip())
        indexToExcludeTest.append(number)

In [75]:
len(indexToExcludeTest)

14150

In [76]:
for i in tqdm(range(len(test_json))):
    
    if i in indexToExcludeTest:
        continue
        
    pointer = test_json[i]
    
    questionList_test.append(pointer['question'])
    answerList_test.append(pointer['answer'])
    imgList_test.append(pointer['image_id'])

100%|██████████████████████████████████████| 86003/86003 [00:07<00:00, 11041.05it/s]


86003 -> 71863

### -------------------------------------- Processing Test Set ----------------------------------------------------------------------------

In [77]:
# check if all the uniques answers are present in the mapping

y,n = 0,0
store = []
for i in range(len(answerList_test)):
    
    word = answerList_test[i]
    
    if word in mapping:
        y += 1
    else:
        n+=1
        store.append(i)

In [78]:
y

71863

In [79]:
labels_test = []

for i in range(len(answerList_test)):
    labels_test.append( mapping[ answerList_test[i] ] )

In [80]:
len(labels_test)

71863

In [81]:
scores_test = []

for i in tqdm(range(len(answerList_test))):
    
    s = [0] * (numOfClasses+1)
    s[ mapping[ answerList_test[i]] ] = 1
    
    scores_test.append(s)

100%|██████████████████████████████████████| 71863/71863 [00:00<00:00, 90940.69it/s]


In [82]:
len(scores_test)

71863

In [83]:
imgPathList_test = []
filepath = '/home/aritra/cric/images/img/'

for i in tqdm(range(len(imgList_test))):
    
    imgName = str(imgList_test[i]) + '.jpg'
    concatedPath = os.path.join(filepath,imgName)
    
    imgPathList_test.append(concatedPath)

100%|█████████████████████████████████████| 71863/71863 [00:00<00:00, 774324.92it/s]


In [84]:
len(imgPathList_test)

71863

In [85]:
imgPathList_test[0:5]

['/home/aritra/cric/images/img/1004.jpg',
 '/home/aritra/cric/images/img/1004.jpg',
 '/home/aritra/cric/images/img/1004.jpg',
 '/home/aritra/cric/images/img/1004.jpg',
 '/home/aritra/cric/images/img/1004.jpg']

In [86]:
# creating HF dataset to map images fast of test_set

listToDictionary = {'questions':questionList_test, 'labels':labels_test, 'scores':scores_test, 'images':imgPathList_test}
modified_test_set = Dataset.from_dict(listToDictionary)

In [87]:
# mapping each filepath of test Set to images in the directory

modified_test_set = modified_test_set.cast_column("images", datasets.Image())

### -------------------------------End of Processing----------------------------------------------------------------------------

In [88]:
from transformers import ViltProcessor, ViltForQuestionAnswering

In [89]:
from transformers import ViltConfig
config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [90]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [91]:
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

In [92]:
model = ViltForQuestionAnswering.from_pretrained("model_chkpts/vilt-mlm-classification-model/vilt_mlm_mod_e4_cric_trained/", id2label = reverse_mapping, label2id = mapping).to(device)

In [93]:
from torch.utils.data import DataLoader
from datasets import Dataset
import numpy as np

In [94]:
class cric_dataset(Dataset):
    
    def __init__(self, dataset, processor):
        self.processor = processor
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,idx):
        
        #print(idx)
        item = self.dataset[idx]

        #print(item)
        
        encodings = self.processor(images = item["images"], text = item["questions"], padding="max_length", truncation=True, return_tensors = "pt")
        encodings = {k:v.squeeze() for k,v in encodings.items()}
                                
        encodings['labels'] = torch.tensor(item['scores'], dtype = torch.float32)
        
        return encodings

In [95]:
train_dataset_object = cric_dataset(modified_train_set, processor)

In [96]:
val_dataset_object = cric_dataset(modified_val_set, processor)

In [97]:
test_dataset_object = cric_dataset(modified_test_set, processor)

In [98]:
color_dataset_object = cric_dataset(train_color_set, processor)

In [99]:
def collate_fn(batch):
  
    input_ids = [item['input_ids'] for item in batch]
    pixel_values = [item['pixel_values'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    token_type_ids = [item['token_type_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
        
    # create padded pixel values and corresponding pixel mask
    
    encoding = processor.image_processor.pad(pixel_values, return_tensors="pt")

    # create new batch
    
    batch = {}
    
    batch['input_ids'] = torch.stack(input_ids)
    batch['attention_mask'] = torch.stack(attention_mask)
    batch['token_type_ids'] = torch.stack(token_type_ids)
    batch['pixel_values'] = encoding['pixel_values']
    batch['pixel_mask'] = encoding['pixel_mask']
    batch['labels'] = torch.stack(labels, dim = 0 )

    return batch


In [100]:
train_dataloader = DataLoader(color_dataset_object, collate_fn = collate_fn, shuffle = True, batch_size = 32)

In [101]:
batch = next(iter(train_dataloader))

In [102]:
for k,v in batch.items():
    print(k, v.shape)
    print()
    
#print(batch.keys())

input_ids torch.Size([32, 40])

attention_mask torch.Size([32, 40])

token_type_ids torch.Size([32, 40])

pixel_values torch.Size([32, 3, 608, 608])

pixel_mask torch.Size([32, 608, 608])

labels torch.Size([32, 1442])



In [103]:
tot_number_of_steps = len(train_dataloader)
tot_number_of_steps

4662

In [104]:
# for fp16 precision

from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler

scaler = GradScaler()

In [105]:
# No Visualisation for this Test Notebook

#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter()

## Model Training Loop

In [108]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5)

model.train()

for epoch in tqdm(range(1)):  

    print(f"Epoch: {epoch}")

    for idx, batch in enumerate(train_dataloader):

        batch = {k:v.to(device) for k,v in batch.items()}

        optimizer.zero_grad()
        
        with torch.autocast(device_type='cuda', dtype=torch.float16):

            outputs = model(**batch)
            loss = outputs.loss

        print(idx,"-> Loss:", loss.item())
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)

        scaler.update()
        
        # Plots in tensorboard
    
        if (idx != 0 ) and (idx % 1000 == 0):
            
            model.eval()
            
            acc_score_test = calculateAccuracyTest()
            acc_score_val, validationLoss = calculateAccuracyVal()
            
            print(f'\nValidation Accuracy: {acc_score_val}, Test Accuracy: {acc_score_test} \n')
            
            #writer.add_scalar('Training Loss', loss.item(), epoch * tot_number_of_steps + idx)
            #writer.add_scalar('Validation Loss', validationLoss, epoch * tot_number_of_steps + idx)

            #writer.add_scalar('Accuracy Score On Val Set', acc_score_val, epoch * tot_number_of_steps + idx)
            #writer.add_scalar('Accuracy Score On Test Set', acc_score_test, epoch * tot_number_of_steps + idx)
            
            model.train()
            
    # Save model checkpoint
    
    save_path = os.path.join('./model_chkpts/test/', 'vilt_mlm_color_e' + '5' + '_cric_trained')
    model.save_pretrained(save_path)
    print("Model Saved At: ", epoch)
    
#writer.close()



  0%|                                                         | 0/1 [00:00<?, ?it/s]

Epoch: 0


/home/aritra/miniconda3/envs/blip_vqa_base_env/lib/python3.8/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343998658/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


0 -> Loss: 0.5371490716934204
1 -> Loss: 0.23276397585868835
2 -> Loss: 0.35655367374420166
3 -> Loss: 0.49922704696655273
4 -> Loss: 0.6733691692352295
5 -> Loss: 0.5433361530303955
6 -> Loss: 1.0520079135894775
7 -> Loss: 0.4814547300338745
8 -> Loss: 0.4572630524635315
9 -> Loss: 0.3161604404449463
10 -> Loss: 0.5157613754272461
11 -> Loss: 0.5291028618812561
12 -> Loss: 0.4964756965637207
13 -> Loss: 0.5185451507568359
14 -> Loss: 0.04080929234623909
15 -> Loss: 0.5630484819412231
16 -> Loss: 0.32479608058929443
17 -> Loss: 0.6960514783859253
18 -> Loss: 0.7995201349258423
19 -> Loss: 0.44206932187080383
20 -> Loss: 0.3656335175037384
21 -> Loss: 0.4552389681339264
22 -> Loss: 0.2756907045841217
23 -> Loss: 0.42417505383491516
24 -> Loss: 0.8510345816612244
25 -> Loss: 0.26363715529441833
26 -> Loss: 0.4432034194469452
27 -> Loss: 0.705394446849823
28 -> Loss: 0.5126279592514038
29 -> Loss: 0.7259231805801392
30 -> Loss: 0.4939831495285034
31 -> Loss: 0.7674992680549622
32 -> Loss:

259 -> Loss: 0.27108097076416016
260 -> Loss: 0.527826189994812
261 -> Loss: 0.5838673710823059
262 -> Loss: 0.7018933296203613
263 -> Loss: 0.14541533589363098
264 -> Loss: 0.45691418647766113
265 -> Loss: 0.4368586540222168
266 -> Loss: 0.13244548439979553
267 -> Loss: 0.2880104184150696
268 -> Loss: 0.4282197952270508
269 -> Loss: 0.21098196506500244
270 -> Loss: 0.2622842490673065
271 -> Loss: 0.7317684292793274
272 -> Loss: 0.40075239539146423
273 -> Loss: 0.8611503839492798
274 -> Loss: 0.29605022072792053
275 -> Loss: 0.21642595529556274
276 -> Loss: 0.6913865804672241
277 -> Loss: 0.4979076683521271
278 -> Loss: 0.3675096333026886
279 -> Loss: 0.9330558180809021
280 -> Loss: 0.3321142792701721
281 -> Loss: 0.44862326979637146
282 -> Loss: 0.4151404798030853
283 -> Loss: 0.36282452940940857
284 -> Loss: 0.4901552200317383
285 -> Loss: 0.06830372661352158
286 -> Loss: 0.7116612195968628
287 -> Loss: 0.8327693343162537
288 -> Loss: 0.7594789266586304
289 -> Loss: 0.574701786041259

...

3983 -> Loss: 0.28351345658302307
3984 -> Loss: 0.45766931772232056
3985 -> Loss: 0.6193770170211792
3986 -> Loss: 0.5360029339790344
3987 -> Loss: 0.2959366738796234
3988 -> Loss: 0.35712164640426636
3989 -> Loss: 0.6055095791816711
3990 -> Loss: 0.16504666209220886
3991 -> Loss: 0.6373705863952637
3992 -> Loss: 0.7779238820075989
3993 -> Loss: 0.8513798117637634
3994 -> Loss: 0.777021586894989
3995 -> Loss: 0.7404495477676392
3996 -> Loss: 0.28082290291786194
3997 -> Loss: 0.7418543100357056
3998 -> Loss: 0.8973482251167297
3999 -> Loss: 0.6786487102508545
4000 -> Loss: 0.4906977117061615

Validation Accuracy: 68.5, Test Accuracy: 73.0 

4001 -> Loss: 1.2212183475494385
4002 -> Loss: 0.381523072719574
4003 -> Loss: 0.3047163188457489
4004 -> Loss: 0.6080929040908813
4005 -> Loss: 0.5316324234008789
4006 -> Loss: 0.5097926259040833
4007 -> Loss: 0.24223873019218445
4008 -> Loss: 0.7684056162834167
4009 -> Loss: 0.4153101146221161
4010 -> Loss: 0.6498619914054871
4011 -> Loss: 0.439891

4229 -> Loss: 0.9669020175933838
4230 -> Loss: 0.4972416162490845
4231 -> Loss: 0.6770495772361755
4232 -> Loss: 0.617667555809021
4233 -> Loss: 0.7423809170722961
4234 -> Loss: 0.3555912673473358
4235 -> Loss: 0.5299884676933289
4236 -> Loss: 0.4626416862010956
4237 -> Loss: 0.6321544647216797
4238 -> Loss: 0.7998590469360352
4239 -> Loss: 0.6336766481399536
4240 -> Loss: 0.7092546224594116
4241 -> Loss: 0.17118629813194275
4242 -> Loss: 0.19824503362178802
4243 -> Loss: 0.566721498966217
4244 -> Loss: 0.45388442277908325
4245 -> Loss: 0.3000195622444153
4246 -> Loss: 0.39488354325294495
4247 -> Loss: 0.33465251326560974
4248 -> Loss: 0.6091470122337341
4249 -> Loss: 0.36915451288223267
4250 -> Loss: 0.8268702626228333
4251 -> Loss: 0.3700551986694336
4252 -> Loss: 0.8854940533638
4253 -> Loss: 0.45941463112831116
4254 -> Loss: 0.41017627716064453
4255 -> Loss: 0.5586594939231873
4256 -> Loss: 0.623632550239563
4257 -> Loss: 0.39537787437438965
4258 -> Loss: 0.13175277411937714
4259 -

4476 -> Loss: 0.5878736972808838
4477 -> Loss: 0.4675315320491791
4478 -> Loss: 0.5279601216316223
4479 -> Loss: 1.0035465955734253
4480 -> Loss: 0.6351914405822754
4481 -> Loss: 0.4929311275482178
4482 -> Loss: 0.11803578585386276
4483 -> Loss: 0.6235116720199585
4484 -> Loss: 0.4695885181427002
4485 -> Loss: 0.41513553261756897
4486 -> Loss: 0.36554983258247375
4487 -> Loss: 0.26845353841781616
4488 -> Loss: 0.641502320766449
4489 -> Loss: 0.5997942090034485
4490 -> Loss: 0.45004355907440186
4491 -> Loss: 0.904541015625
4492 -> Loss: 0.5891513228416443
4493 -> Loss: 0.35151320695877075
4494 -> Loss: 0.24684825539588928
4495 -> Loss: 0.5392006635665894
4496 -> Loss: 0.8868497610092163
4497 -> Loss: 0.6278976798057556
4498 -> Loss: 0.41750213503837585
4499 -> Loss: 0.7766048312187195
4500 -> Loss: 0.5278530120849609
4501 -> Loss: 0.8324658870697021
4502 -> Loss: 0.1490786075592041
4503 -> Loss: 0.41637587547302246
4504 -> Loss: 0.04900688678026199
4505 -> Loss: 0.654534101486206
4506 -

100%|█████████████████████████████████████████████| 1/1 [1:35:05<00:00, 5705.46s/it]

Model Saved At:  0


In [ ]:
model.eval()

In [ ]:
index = 1

In [ ]:
example = val_dataset_object[index]
print(example.keys())

In [ ]:
delLab = example.pop('labels')

In [ ]:
processor.decode(example['input_ids'])

In [ ]:
# add batch dimension + move to GPU
example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}

# forward pass
outputs = model(**example)

In [ ]:
logits = outputs.logits
logits.shape

In [ ]:
logits

In [ ]:
print(torch.argmax(logits).item())
reverse_mapping[logits.argmax(-1).item()]

In [ ]:
answerList[index]

In [ ]:
predicted_classes = torch.sigmoid(logits)
probs, classes = torch.topk(predicted_classes, 5)

for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
  print(prob, model.config.id2label[class_idx])

In [ ]:
i = Image.open(imgPathList[index])
i.thumbnail((300,300))
i

## Reports & Results

In [106]:
# This function returns the Validation Loss and accuracy on the Validation Set

def calculateAccuracyVal():
    
    matchScore, loopCounter = 0,0
    
    for index in range(0,200):
        
        loopCounter += 1
        
        val_example = val_dataset_object[index]
        val_example = {k: v.unsqueeze(0).to(device) for k,v in val_example.items()}
        val_outputs = model(**val_example)
        
        validationLoss = val_outputs.loss

        val_logits = val_outputs.logits
        val_predicted_classes = torch.sigmoid(val_logits)
        val_ans = reverse_mapping[torch.argmax(val_predicted_classes).item()]
        
        
        # accuracy score
        
        if answerList_val[index] == val_ans:
            matchScore += 1
                
    #print(matchScore, loopCounter)
    accuracyVal = (matchScore/loopCounter)*100
    return ( accuracyVal,validationLoss.item() )

In [ ]:
calculateAccuracyVal()

In [107]:
# This function returns accuracy on the Test Set

def calculateAccuracyTest():
    
    matchScore, loopCounter = 0,0
    model.eval()
    for index in range(0, 200):
        
        loopCounter += 1
        
        test_example = test_dataset_object[index]
        test_example = {k: v.unsqueeze(0).to(device) for k,v in test_example.items()}
        test_outputs = model(**test_example)

        test_logits = test_outputs.logits
        test_predicted_classes = torch.sigmoid(test_logits)
        test_ans = reverse_mapping[torch.argmax(test_predicted_classes).item()]
        
        # print(f'T: {answerList_val[index]} <-> P: {test_ans}' )

        # accuracy score
        
        if answerList_test[index] == test_ans:
            matchScore += 1
                
    #print(matchScore, loopCounter)
    return ((matchScore/loopCounter)*100)

In [ ]:
calculateAccuracyTest()

In [ ]:
# This function returns report on the Test Set

misclassifiedIndex = []
def generateReport():
    
    matchScore, loopCounter = 0,0
    model.eval()
    
    for index in range(0,500):
        
        loopCounter += 1
        print(f'\n{questionList_test[index]} ? Ans: {answerList_test[index]}\n')
        
        example = test_dataset_object[index]
        example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}
        outputs = model(**example)

        logits = outputs.logits
        predicted_classes = torch.sigmoid(logits)
        ans = reverse_mapping[torch.argmax(predicted_classes).item()]
        
        print('Predicted Ans:', ans,'\n')
        
        probs, classes = torch.topk(predicted_classes, 4)

        for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
            print(prob, model.config.id2label[class_idx])
    
        # accuracy score
        
        if answerList_test[index] == ans:
            matchScore += 1
            print('Correct Prediction at index:', index)
        
        else:
            misclassifiedIndex.append(index)
            print('Wrong Prediction at index:', index)
    
    return ((matchScore/loopCounter)*100)

In [ ]:
generateReport()

In [ ]:
misclassifiedIndex

In [ ]:
i = Image.open(imgPathList_test[53054])
i.thumbnail((500,500))
i

## Find Questions With Color And Its Accuracy

In [ ]:
# extracting the list of colors from the previously stored text files

colors = []
with open('./text_files/colors.txt', 'r') as file:
    for color in file:
        color = color.strip()
        colors.append(color)

In [ ]:
colors[0:5]

In [ ]:
# adding leading and trailing space in the colors

colors_spaces = [' '+ color + ' ' for color in colors] 

In [ ]:
colors_spaces[0:5]

In [ ]:
def isContainColor(targetString, colorList):
    
    for color in colorList:
        if color in targetString:
            return True
    
    return False    

In [ ]:
# This function finds the accuracy on color question and identifies the color question for which the result is misclassified

misclassifiedIndex = []
colorFrequency = {color: 0 for color in colors}

def findColorQuestions():
    
    global colors
    matchScore, questionCount = 0,0
    model.eval()
    
    print('***** Question About Colors ************')
    
    for index in tqdm(range(1000,2000)):
        
        currQuestion = questionList_test[index]        
        
        if ('color' in currQuestion) or (isContainColor(currQuestion, colors_spaces)):
            
            questionCount += 1
            
            #print(f'\n{questionList_test[index]} ? Ans: {answerList_test[index]}\n')
            
            example = test_dataset_object[index]
            example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}
            outputs = model(**example)

            logits = outputs.logits
            predicted_classes = torch.sigmoid(logits)
            ans = reverse_mapping[torch.argmax(predicted_classes).item()]

            # accuracy score

            if answerList_test[index] == ans:
                matchScore += 1

            else:
                
                if answerList_test[index] in colors:
                    colorFrequency[ answerList_test[index] ] = colorFrequency[ answerList_test[index] ] + 1
                    
                misclassifiedIndex.append(index)
                                
        else:
            
            continue
    
    
    print(f'\nTotal {questionCount} questions found')
    return ((matchScore/questionCount)*100)

In [ ]:
findColorQuestions()

In [ ]:
len(misclassifiedIndex)

In [ ]:
colorFrequency

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
categories = colorFrequency.keys()
values = colorFrequency.values()

In [ ]:
plt.figure(figsize=(14, 7))

plt.xlabel(f"Model Misclassified Total {sum(values)} Questions involving Colors")
plt.ylabel("Values")
plt.title("Number of times Ground truth Colors which has been misclassified by the model")

plt.bar(colorFrequency.keys(), colorFrequency.values(), color='lightblue', edgecolor='black', width=0.4)
plt.grid(True)

# Show the plot
plt.show()

## Store Color Question Indices

In [ ]:
# extracting the list of colors from the previously stored text files

colors_train = []
with open('./text_files/colors_train.txt', 'r') as file:
    for color in file:
        color = color.replace("\n","")
        colors_train.append(color)

In [ ]:
# This function collects the indices of the color questions from the train set

colorQuestionIndices = []

def storeColorQuestionIndex():
    
    questionCount = 0 
    print('********* Storing Color Questions Indices ************')
    
    for index in tqdm(range(0,len(answerList))):
        
        currAnswer = answerList[index]  
        currQuestion = questionList[index]
                
        if ('color' in currQuestion) or (isContainColor(currQuestion,colors_train)):
            #print(index,currQuestion)

            questionCount += 1
            colorQuestionIndices.append(index)
        
                
    print(f'\nTotal {questionCount} color questions found')

In [ ]:
storeColorQuestionIndex()

In [ ]:
len(colorQuestionIndices)

In [ ]:
colorQuestionIndices[1000:1005]

In [ ]:
questionList[2560]

## Words in color Questions Histogram

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
def remove_stopwords(wordList):
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in wordList if word.lower() not in stop_words]
    return filtered_words

In [ ]:
# this function gathers all the words from the color questions and their frequency from all the color questions to make histogram

frquencyMap = {}

def collectWords():
    
    for index in tqdm(misclassifiedIndex):
        
        currQuestion = questionList_test[index]
        words = remove_stopwords(currQuestion.split())
        
        for word in words:
            
            if word in frquencyMap:
                
                frquencyMap[word] = frquencyMap[word] + 1
            
            else:
            
                frquencyMap[word] = 1


In [ ]:
collectWords()

In [ ]:
len(frquencyMap)

In [ ]:
# convert to list of tuples

frquencyList = [(key,val) for key,val in frquencyMap.items()]

In [ ]:
frquencyList[0:5]

In [ ]:
frquencyList.sort(key = lambda x:x[1], reverse = True)

In [ ]:
frquencyList[0:5]

In [ ]:
import seaborn as sns

In [ ]:
# top-30 words

frquencyList = frquencyList[0:30]
labels = [ val[0] for val in frquencyList]
frequncies = [ val[1] for val in frquencyList]

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=frequncies, y=labels, palette="viridis")

# Customize the plot
ax.set(xlabel="Frequency", ylabel="Words", title="Word Frequency")
plt.tight_layout()

plt.show()

# Experimenting With Accuracy By Removing Most Frequent Words

In [ ]:
len(questionList_test)

In [ ]:
for index in tqdm(range(0,500)):

    currQuestion = questionList_test[index]
    if 'object' or 'used' in currQuestion:
        currQuestion = ' '.join([word for word in currQuestion.split() if word not in ('object','used')])
    
    questionList_test[index] = currQuestion

# creating HF dataset to map images fast of test_set

listToDictionary = {'questions':questionList_test, 'labels':labels_test, 'scores':scores_test, 'images':imgPathList_test}
word_removed_test_set = Dataset.from_dict(listToDictionary)

In [ ]:
word_removed_test_set = word_removed_test_set.cast_column("images", datasets.Image())

In [ ]:
word_removed_test_set[10]['questions'], modified_test_set[10]['questions']

In [ ]:
word_removed_test_set_object = cric_dataset(word_removed_test_set, processor)

In [ ]:
# This function returns report on the Test Set

misclassifiedIndex = []
def removeWordsAndGenReport():
    
    matchScore, loopCounter = 0,0
    model.eval()
    
    for index in tqdm(range(0,1000)):
        
        loopCounter += 1                            

        example = word_removed_test_set_object[index]
        example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}
        outputs = model(**example)

        logits = outputs.logits
        predicted_classes = torch.sigmoid(logits)
        ans = reverse_mapping[torch.argmax(predicted_classes).item()]
        
        # print('Predicted Ans:', ans,'\n')
        
        probs, classes = torch.topk(predicted_classes, 4)

        for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
            print(end='')
    
        # accuracy score
        
        if answerList_test[index] == ans:
            matchScore += 1
            #print('Correct Prediction at index:', index)
        
        else:
            misclassifiedIndex.append(index)
            #print('Wrong Prediction at index:', index)
    
    return ((matchScore/loopCounter)*100)

In [ ]:
removeWordsAndGenReport()

In [ ]:
len(questionList)

In [ ]:
pwd

In [ ]:
cat text_files/colors_train.txt

In [ ]:
# find the color questions of the training set

# This function identifies the color question for which the result is misclassified

colorMap = {}
questionCount = 0

def findColorQuestionsTraining():
        
    print('***** Question About Colors ************')
    
    for index in tqdm(range(len(questionList))):
        
        currQuestion = questionList[index]        
        #print(currQuestion)
        
        if ('color' in currQuestion):
            
            global questionCount
            questionCount += 1
            currAnswer = answerList[index]
            
            if currAnswer in colorMap:              
                colorMap[currAnswer] = colorMap[currAnswer] + 1
            else:
                colorMap[currAnswer] = 1

            
        else:
            
            continue
            
    return (colorMap)

In [ ]:
findColorQuestionsTraining()